In [1]:
def interp(data):
    '''
    ### Interpolation of data containing Nan's ###
    
    input: list (n,) with data
    output: ndarray (n,) with interpolated data 
    
    '''
    data = np.reshape(data,(len(data),)) #reshaping list
    nan = ~np.isnan(data) #all elements that are not nan 
    idx_nonzero = nan.ravel().nonzero()[0] #flattens the array and returns the indices that are nonzero 
    y = data[~np.isnan(data)] #y-coordinates of the data points that are not nan
    x  = np.isnan(data).ravel().nonzero()[0] #x-coordinates that evaluates interpolation
    #print('x=',x)
    #print('idx non zero=', idx_nonzero)
    #print('y=',y)
    
    if len(x) == 0: 
        data = data
        
    else:
        data[np.isnan(data)] = np.interp(x, idx_nonzero, y) #interpolating the data 
    
    return data

In [2]:
import warnings 
def annual_average(annual, age, data):
    '''
    ### Annual average of any ice core data with corresponding thickness ###
    
    Input: 
    annual: ndarray (n,) with annual integers.
    age: ndarray (m,) with real age data.
    data: ndarray (m,) with sulfate or sulfur data.
    thickness: ndarray (m-1,) with thickness of each data point.
    
    Output:
    annual_data: array (n,) with annual sulfate or sulphur data.
    annual_thickness: array (n,) with annual thickness corresponding
    to the annual sulphur or sulfate layers.
    
    ''' 
    
    annual_data = []

    
    for i in range(len(annual)):
        index = np.where((age > annual[i]) & (age < annual[i]+1))
        
        if data[index].size == 0:
            annual_data.append(np.nan)
  
        else:

            with warnings.catch_warnings(): #ignoring sums/mean of only nan's
                warnings.simplefilter("ignore", category=RuntimeWarning)

                if index[0][-1] == (len(data)-1) or index[0][0] == (len(data)-1): #NEED ANOTHER SOLUTION HERE
                
                    index = index[0][:-1]
                    annual_data.append(np.nanmean(data[index]))
                
                else:
    
                    annual_data.append(np.nanmean(data[index]))
                    
    return np.array(annual_data)

In [3]:
def running_median(annual, data, y):
    '''
    ### Running median over an annual averaged data set ###
    input: 
    annual: ndarray (n,) with annual integers.
    data: ndarray (n,) with annualy averaged data.
    y: even integer representing the length of the median filter. 
    
    output:
    RM: ndarray (n,) with the y year running median of the data set. 
    '''
    #RM = scipy.signal.medfilt(data, kernel_size=y)
    RM = np.zeros((len(annual)))
    
    for i in range(len(annual)):
        with warnings.catch_warnings(): #ignoring sums/mean of only nan's
                warnings.simplefilter("ignore", category=RuntimeWarning)

                idx = np.where((annual> annual[i]-((y+1)/2)) & (annual<annual[i]+((y+1)/2)) & (~np.isnan(annual)))
                RM[i] = np.nanmedian(data[idx])

    return RM
     
def median_of_absolute_deviation(annual, data, y, z=3):
    '''
    ### Running median of absolute deviation * 3 + the running median (31 years) ###
    input: 
    annual: ndarray (n,) with annual integers.
    data: ndarray (n,) with annualy averaged data.
    y: even integer representing the length of the median filter. 
    z: integer which decides the volcanic threshold. (default=3)
    
    output:
    y: ndarray (n,) with z*MAD + RM. 
    '''
    RM = np.zeros((len(annual)))
    RMAD = np.zeros((len(annual)))
    #y = np.zeros((len(annual)))
    #absolute = np.zeros((len(annual)))
    
    for i in range(len(annual)):

        idx = np.where((annual> annual[i]-((y+1)/2)) & (annual<annual[i]+((y+1)/2)) & (~np.isnan(annual))) #index
        
        if np.shape(data[idx]) == 0:
            RMAD = np.nan 
            RM = np.nan
        else: 
            with warnings.catch_warnings(): #ignoring sums/mean of only nan's
                warnings.simplefilter("ignore", category=RuntimeWarning)
                running_median = np.nanmedian(data[idx]) #float
                absolute = np.abs(data[idx] - running_median) #array
                RM[i] = running_median #float
                #RMAD[i] = np.nanmedian(absolute) #float
                RMAD[i] = scipy.stats.median_abs_deviation(data[idx], axis=0, center = np.median,scale=1.0, nan_policy = 'omit')
    y = z*RMAD + RM
    return y

In [4]:
def exl_volcanism(annual, data,y,z):
    '''
    ### Calculates the annual data without influence of volcanism ###
    input:
    annual: ndarray (n,) with annual integers.
    data: ndarray (n,) with annualy averaged data.
    y: even integer representing the length of the median filter. 
    z: integer which decides the volcanic threshold. (default=3)
    
    output:
    data_interp: ndarray (n,) with annual data without influence of volcanism.
    '''
    limit = median_of_absolute_deviation(annual,data,y,z)
    data_mod = np.copy(data)
    
    for i in range(len(data)):
        if data_mod[i] > limit[i]:
            data_mod[i] = np.nan
        else:
            continue
    
    data_interp = interp(data_mod)
    
    return data_mod

In [5]:
def sulfate_deposition(annual,data,thickness,thinning,l,s):
    '''
    ### Calculates the sulfate deposition within a peak and its duration ###
    input:
    annual: ndarray (n,) with annual integers.
    data: ndarray (n,) with annualy averaged data.
    thickness: ndarray (n,) with annual thickness.
    thinning: ndarray(n,) with annual thinning function.
    l: even integer representing the length of the median filter.
    s: integer; 1 if data is sulfate, 3 if data is sulfur. 
     
    output:
    sulfate_dep: list (b,) with total sulfate deposition = volcanic event.  
    first_year: list (b,) with start of sulfate peak. 
    last_year: list(b,) with end of sulfate peak. 
    length: list (b,) with the length of each eruption.
    
    (b is the number of volcanic peaks found in the ice core.)
    '''
    sulfate = []
    sulfate_dep = []
    year = []
    first_year = []
    last_year = []
    length = []
    
    exl_volc = exl_volcanism(annual,data,l)
    
    RM = running_median(annual,exl_volc,l)
    y = median_of_absolute_deviation(annual,exl_volc,l)
    
    for i in range(len(data)):
        
        if data[i] > y[i]:
            sulfate.append(( ( (data[i] - RM[i]) * s) * thickness[i]/thinning ) * 1e-3 * 917)
            year.append(annual[i])
            
        else:
            if len(sulfate) != 0:
                sulfate_dep.append(np.sum(sulfate))
                first_year.append(year[0])
                last_year.append(year[-1])
                length.append(len(year))
                
                sulfate = []
                year=[]
            else:
                sulfate=[]
                year = []
        
    return sulfate_dep, first_year, last_year, length